<a href="https://colab.research.google.com/github/lucia1299/DBMS_InvestigativeJournalism/blob/main/mREBEL_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the relevant libraries

In [ ]:
!pip install transformers wikidata requests rdflib SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import IPython
from wikidata.client import Client
import requests
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON

# Tryout with Wikidata

In [ ]:
    def get_wikidata_data(candidate_entity):
        client = Client()
        search_url = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbsearchentities',
            'format': 'json',
            'language': 'it',
            'search': candidate_entity,
            'limit': 1
        }

        try:
            # Search for the entity using the Wikidata API
            response = requests.get(search_url, params=params)
            search_results = response.json()

            if not search_results.get('search'):
                return None

            # Get the first matched entity's ID
            entity_id = search_results['search'][0]['id']

            # Fetch the entity data using the Wikidata client
            wikidata_entity = client.get(entity_id, load=True)

            title = wikidata_entity.label.get('en')

            entity_data = {
                "title": title,
                "url": f"https://www.wikidata.org/wiki/{entity_id}",
                # You can also extract more data as needed
            }

            return entity_data
        except Exception as e:
            print(f"Error fetching data: {e}")
            return None

In [ ]:
entity_data = get_wikidata_data("Cina")
if entity_data:
    print(f"Title: {entity_data['title']}")
    print(f"URL: {entity_data['url']}")
else:
    print("Entity not found or error occurred.")

Title: People's Republic of China
URL: https://www.wikidata.org/wiki/Q148


# Tryout with LOV query


In [ ]:
import requests

def search_lov_property(prop_name):
    base_url = "https://lov.linkeddata.es/dataset/lov/api/v2/term/search"

    params = {
        "q": prop_name,
        "type": "property"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

property_name = "country of citizenship"

def get_lov_property(property_name):
    results = search_lov_property(property_name)
    if results:
        max_score = float('-inf')
        result_with_max_score = None

        for result in results.get("results", []):
            score = result.get('score', float('-inf'))

            if score > max_score:
                max_score = score
                result_with_max_score = result

            if result_with_max_score:
                return str(result_with_max_score.get('uri', 'N/A')[0])
            else:
                print("No results found or an error occurred.")

final = get_lov_property(property_name)
print(final)
print(type(final))

http://contextus.net/ontology/ontomedia/ext/common/being#place_of_citizenship
<class 'str'>


# Tryout with SemanticTripleConverter

In [ ]:
class SemanticTripleConverter:
    def __init__(self, entities, relations):
        self.entities = entities
        self.relations = relations
        self.sparql_endpoint = "https://dbpedia.org/sparql"
        self.ontology_predicates = {
            'is': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
        }

    def get_valid_predicate(self, predicate):
        # First, check initial mappings
        if predicate in self.ontology_predicates:
            return self.ontology_predicates[predicate]

        # Second, query external SPARQL endpoint
        external_predicate = self.query_external_ontology(predicate)
        if external_predicate:
            # Cache the result to avoid future queries
            self.ontology_predicates[predicate] = external_predicate
            return external_predicate

        # Fallback to original predicate if no match found
        return predicate

    def search_lov_property(prop_name):
        base_url = "https://lov.linkeddata.es/dataset/lov/api/v2/term/search"

        params = {
            "q": prop_name,
            "type": "property"
        }

        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            return response.json()
        else:
            return None

    def get_lov_property(property_name):
        results = search_lov_property(property_name)
        if results:
            max_score = float('-inf')
            result_with_max_score = None

            for result in results.get("results", []):
                score = result.get('score', float('-inf'))

                if score > max_score:
                    max_score = score
                    result_with_max_score = result

                if result_with_max_score:
                    return str(result_with_max_score.get('uri', 'N/A')[0])
                return None

    def query_external_ontology(self, predicate):
        sparql = SPARQLWrapper(self.sparql_endpoint)
        query = f"""
        SELECT ?property ?label
        WHERE {{
        ?property a rdf:Property .
        ?property rdfs:label ?label .
        FILTER (lang(?label) = "en" && CONTAINS(LCASE(?label), LCASE("{predicate}")))
        }}
        """
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        if results["results"]["bindings"]:
            return results["results"]["bindings"][0]["property"]["value"]
        else:
            lov_predicate = get_lov_property(predicate)
            return lov_predicate

    def convertJSON(self):
        triples = []
        for entity in self.entities.keys():
            triple = {
                'subject': entity,
                'predicate': str(self.get_valid_predicate('is')),
                'object': self.entities[entity]['url']
            }
            triples.append(triple)
        for relation in self.relations:
            triple = {
                'subject': relation['head'],
                'predicate': str(self.get_valid_predicate(relation['type'])),
                'object': relation['tail']
            }
            triples.append(triple)
        return triples

# Example usage:
entities = {
    'Serbia': {'url': 'http://www.wikidata.org/entity/Q403'}
}
relations = [
    {'head': 'Serbia', 'type': 'diplomatic relation', 'tail': 'European Union'},
    {'head': 'Serbia', 'type': 'head of state', 'tail': 'Aleksandar Vučić'},
    {'head': 'Serbia', 'type': 'country of citizenship', 'tail': 'Mihajlo Pupin'}
]

converter = SemanticTripleConverter(entities, relations)
triples = converter.convertJSON()

for triple in triples:
    print(triple)

{'subject': 'Serbia', 'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'object': 'http://www.wikidata.org/entity/Q403'}
{'subject': 'Serbia', 'predicate': 'http://purl.org/dc/elements/1.1/relation', 'object': 'European Union'}
{'subject': 'Serbia', 'predicate': 'http://dbpedia.org/property/headOfState', 'object': 'Aleksandar Vučić'}
{'subject': 'Serbia', 'predicate': 'http://contextus.net/ontology/ontomedia/ext/common/being#place_of_citizenship', 'object': 'Mihajlo Pupin'}
